# Flights Dataset
## by Faisal AlDhuwayhi

## Preliminary Wrangling

> This dataset reports flights of California state, of the year of 2019. Flights that are going into California or going outside of California. Including variables like destinations, arrival and departure delays, and more other features.

In [171]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Loading in your dataset and figuring its properties.

In [208]:
# import the data
import os

flights = pd.DataFrame()
for month in os.listdir('./dataset_flights_2019/California/'):
    if month != '.DS_Store':
        temp_df = pd.read_csv('./dataset_flights_2019/California/'+month)
        flights = flights.append(temp_df, ignore_index=True)

In [209]:
# drop a defective columns 
flights.drop('Unnamed: 17', axis=1, inplace=True)

In [210]:
# let's look at the size of the dataset
flights.shape

(1379233, 17)

In [211]:
# have a general look at it 
flights.head()

,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_CITY_NAME,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE
0,2,4,1,1,"San Diego, CA","Las Vegas, NV",650,638.0,-12.0,815,748.0,-27.0,0.0,NaN,85.0,70.0,258.0
1,2,4,2,2,"San Diego, CA","Las Vegas, NV",650,641.0,-9.0,815,800.0,-15.0,0.0,NaN,85.0,79.0,258.0
2,2,4,3,3,"San Diego, CA","Las Vegas, NV",650,645.0,-5.0,815,811.0,-4.0,0.0,NaN,85.0,86.0,258.0
3,2,4,4,4,"San Diego, CA","Las Vegas, NV",650,644.0,-6.0,815,756.0,-19.0,0.0,NaN,85.0,72.0,258.0
4,2,4,5,5,"San Diego, CA","Las Vegas, NV",650,651.0,1.0,815,814.0,-1.0,0.0,NaN,85.0,83.0,258.0


In [212]:
# properties of the dataset
flights.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379233 entries, 0 to 1379232
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   QUARTER              1379233 non-null  int64  
 1   MONTH                1379233 non-null  int64  
 2   DAY_OF_MONTH         1379233 non-null  int64  
 3   DAY_OF_WEEK          1379233 non-null  int64  
 4   ORIGIN_CITY_NAME     1379233 non-null  object 
 5   DEST_CITY_NAME       1379233 non-null  object 
 6   CRS_DEP_TIME         1379233 non-null  int64  
 7   DEP_TIME             1358922 non-null  float64
 8   DEP_DELAY            1358921 non-null  float64
 9   CRS_ARR_TIME         1379233 non-null  int64  
 10  ARR_TIME             1357678 non-null  float64
 11  ARR_DELAY            1355040 non-null  float64
 12  CANCELLED            1379233 non-null  float64
 13  CANCELLATION_CODE    20859 non-null    object 
 14  CRS_ELAPSED_TIME     1379233 non-null  float64
 15

In [213]:
# we need to clean some issue on the data
flights_master = flights.copy()

In [214]:
# check duplicates and remove it
print("Before removing duplicates:", flights_master.duplicated().sum())

flights_master = flights_master[~flights_master.duplicated()]

print("After removing duplicates:", flights_master.duplicated().sum())

Before removing duplicates: 6
After removing duplicates: 0


In [215]:
# make all the columns names as lowercase to make the indexing easier
flights_master.columns = [column.lower() for column in flights_master.columns.tolist()]
flights_master.columns

Index(['quarter', 'month', 'day_of_month', 'day_of_week', 'origin_city_name',
       'dest_city_name', 'crs_dep_time', 'dep_time', 'dep_delay',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'crs_elapsed_time', 'actual_elapsed_time',
       'distance'],
      dtype='object')

In [216]:
# split the city and the state in different columns

# origin state and city
origin_city_state = flights_master.origin_city_name.str.split(', ')
flights_master['origin_city_name'] = origin_city_state.apply(lambda x : x[0])
origin_state_name = origin_city_state.apply(lambda x : x[1])

index = flights_master.columns.get_loc("origin_city_name")
flights_master.insert(loc=index+1, column='origin_state_name', value=origin_state_name)


# destination state and city
dest_city_state = flights_master.dest_city_name.str.split(', ')
flights_master['dest_city_name'] = dest_city_state.apply(lambda x : x[0])
dest_state_name = dest_city_state.apply(lambda x : x[1])

index = flights_master.columns.get_loc("dest_city_name")
flights_master.insert(loc=index+1, column='dest_state_name', value=dest_state_name)

In [217]:
flights_master.sample()

,quarter,month,day_of_month,day_of_week,origin_city_name,origin_state_name,dest_city_name,dest_state_name,crs_dep_time,dep_time,dep_delay,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance
53901,2,4,3,3,Las Vegas,NV,Los Angeles,CA,1700,1651.0,-9.0,1815,1813.0,-2.0,0.0,NaN,75.0,82.0,236.0


In [218]:
# change the cancellation_code to the real variable and make it as a category variable

# this conversion is from the documentation of the dataset
conversions = {'A' : 'Carrier', 'B' : 'Weather', 'C' : 'National Air System', 'D' : 'Security'}

flights_master['cancellation_code'] = flights_master.cancellation_code.apply(lambda x : conversions.get(x, np.nan))
flights_master['cancellation_code'] = flights_master['cancellation_code'].astype('category')

In [219]:
flights_master.cancellation_code.value_counts()

Carrier                10608
Weather                 7127
National Air System     3114
Security                   8
Name: cancellation_code, dtype: int64

In [220]:
# change of day_of_week to the names of the days and make it categorical
import calendar

flights_master['day_of_week'] = flights_master.day_of_week.apply(lambda x : calendar.day_name[x-1])

categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dtype = pd.api.types.CategoricalDtype(categories=categories, ordered=True)
flights_master['day_of_week'] = flights_master['day_of_week'].astype(dtype)

In [221]:
flights_master.day_of_week.value_counts()

Monday       206100
Friday       205623
Thursday     203987
Tuesday      201032
Wednesday    200310
Sunday       194952
Saturday     167223
Name: day_of_week, dtype: int64

In [222]:
# change of month numbers to the names of the months and make it categorical
flights_master['month'] = flights_master.month.apply(lambda x : calendar.month_name[x])

categories = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October',
              'November', 'December']
dtype = pd.api.types.CategoricalDtype(categories=categories, ordered=True)
flights_master['month'] = flights_master['month'].astype(dtype)

In [223]:
flights_master.month.value_counts()

August       125957
July         124471
June         118327
October      117693
December     116963
May          116042
March        114859
September    112764
April        112191
November     111026
January      109798
February      99136
Name: month, dtype: int64

In [224]:
# change the quarters to be categorical variable
flights_master['quarter'] = flights_master['quarter'].astype(str)
conversions = {'1' : 'First', '2' : 'Second', '3' : 'Third', '4' : 'Forth'}
flights_master['quarter'] = flights_master.quarter.apply(lambda x : conversions[x])

categories = ['First', 'Second', 'Third', 'Forth']
dtype = pd.api.types.CategoricalDtype(categories=categories, ordered=True)
flights_master['quarter'] = flights_master['quarter'].astype(dtype)

In [225]:
flights_master.quarter.value_counts()

Third     363192
Second    346560
Forth     345682
First     323793
Name: quarter, dtype: int64

In [226]:
# Remove the nan values
flights_master.drop(flights_master[flights_master.actual_elapsed_time.isna()].index, inplace=True)

In [227]:
# convert the types of some variables to integer
flights_master['dep_time'] = flights_master.dep_time.astype(int)
flights_master['dep_delay'] = flights_master.dep_delay.astype(int)
flights_master['arr_time'] = flights_master.arr_time.astype(int)
flights_master['arr_delay'] = flights_master.arr_delay.astype(int)
flights_master['crs_elapsed_time'] = flights_master.crs_elapsed_time.astype(int)
flights_master['actual_elapsed_time'] = flights_master.actual_elapsed_time.astype(int)
flights_master['cancelled'] = flights_master.cancelled.astype(int)

In [228]:
flights_master.dtypes

quarter                category
month                  category
day_of_month              int64
day_of_week            category
origin_city_name         object
origin_state_name        object
dest_city_name           object
dest_state_name          object
crs_dep_time              int64
dep_time                  int64
dep_delay                 int64
crs_arr_time              int64
arr_time                  int64
arr_delay                 int64
cancelled                 int64
cancellation_code      category
crs_elapsed_time          int64
actual_elapsed_time       int64
distance                float64
dtype: object

In [229]:
# convert hhmm format to time format hh:mm:ss, and also keeping the original
def str_to_time(time):
    time = str(time)
    new_time = ''
    if len(time) == 1:
        new_time = "00:0{}:00".format(time)
    elif len(time) == 2:
        new_time = "00:{}:00".format(time)
    elif len(time) == 3:
        timeh = time[0]
        timem = time[1:]
        new_time = "0{}:{}:00".format(timeh, timem)
    elif len(time) == 4:
        if time != '2400':
            timeh = time[0:2]
            timem = time[2:]
            new_time = "{}:{}:00".format(timeh, timem)
        else:
            new_time = "00:00:00"
        
    return new_time

In [230]:
crs_dep_time_new = flights_master.crs_dep_time.apply(lambda x : str_to_time(x))
dep_time_new = flights_master.dep_time.apply(lambda x : str_to_time(x))
crs_arr_time_new = flights_master.crs_arr_time.apply(lambda x : str_to_time(x))
arr_time_new = flights_master.arr_time.apply(lambda x : str_to_time(x))

index = flights_master.columns.get_loc("crs_dep_time")
flights_master.insert(loc=index+1, column='crs_dep_time_new', value=crs_dep_time_new)

index = flights_master.columns.get_loc("dep_time")
flights_master.insert(loc=index+1, column='dep_time_new', value=dep_time_new)

index = flights_master.columns.get_loc("crs_arr_time")
flights_master.insert(loc=index+1, column='crs_arr_time_new', value=crs_arr_time_new)

index = flights_master.columns.get_loc("arr_time")
flights_master.insert(loc=index+1, column='arr_time_new', value=arr_time_new)

In [231]:
# reset the row index
flights_master.index = range(flights_master.shape[0])

### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!